## Evaluación de resultados

In [1]:
MODEL = "bert-clinical-scratch-wl-es-NER-prescription"

In [2]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [3]:
model = AutoModelForTokenClassification.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [4]:
from auxfunctions import eval_text, map_entities

In [5]:
text = "PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias"
# text = text.split()

eval_text(text,tokenizer,model)

array([ 1,  0,  0,  3,  0,  0,  5,  7,  8,  8,  9, 10, 10])

In [6]:
ner_dict = {'O': 0,
            'B-ACTIVE_PRINCIPLE': 1,
            'I-ACTIVE_PRINCIPLE': 2,
            'B-FORMA_FARMA':3,
            'I-FORMA_FARMA':4,
            'B-ADMIN': 5,
            'I-ADMIN': 6,
            'B-PERIODICITY': 7,
            'I-PERIODICITY': 8,
            'B-DURATION': 9,
            'I-DURATION': 10
            }

print(text)
map_entities(eval_text(text,tokenizer,model),ner_dict)

PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias


['B-ACTIVE_PRINCIPLE',
 'O',
 'O',
 'B-FORMA_FARMA',
 'O',
 'O',
 'B-ADMIN',
 'B-PERIODICITY',
 'I-PERIODICITY',
 'I-PERIODICITY',
 'B-DURATION',
 'I-DURATION',
 'I-DURATION']

### Conjuntos test

In [7]:
import sys
sys.path.append('../datos/procesamiento')
from corpus import Corpus

datos_conll = Corpus()

In [8]:
datos_conll.entidades = ner_dict

In [9]:
for i in range(4):
    datos_conll.load_conll('../datos/Etiquetado/corpus_s{}_etiquetados.conll'.format(i+1))

print(len(datos_conll))

Agregadas 250 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus
1003


In [12]:
HF_data_mini = datos_conll.to_HF_dataset()

HF_dataset_mini = HF_data_mini.train_test_split(test_size=0.2,seed=0)
HF_dataset_mini

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 802
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 201
    })
})

In [13]:
from auxfunctions import tokenize_and_align_labels
process = lambda examples: tokenize_and_align_labels(examples,tokenizer)

tokenized_data_mini = HF_dataset_mini.map(process, batched=True)
tokenized_data_mini = tokenized_data_mini.remove_columns(['id','tokens','ner_tags'])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from auxfunctions import eval_text, map_entities, calculate_metrics

y_test = [row['ner_tags'] for row in HF_dataset_mini['test']]
y_preds = [list(eval_text(row['tokens'],tokenizer,model)) for row in HF_dataset_mini['test']]

calculate_metrics(y_preds,y_test,ner_dict=ner_dict)

Resultados de evaluación
	 f1: 0.50 | precision: 0.53 | recall: 0.47


(0.5299600532623169, 0.4654970760233918, 0.49564134495641343)

**Test basado en ER**

In [17]:
corpus_test = Corpus()
corpus_test.entidades = ner_dict

corpus_test.load_conll('../datos/Etiquetado/corpus_ER_test.txt')
print(len(corpus_test))

Agregadas 51183 secuencias de token-entidad al corpus
154552


In [ ]:
HF_test = corpus_test.to_HF_dataset()
HF_test